In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

gender_submission.csv
test.csv
train.csv



In [2]:
#訓練データ読み込み
df = pd.read_csv("../input/train.csv")
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [3]:
#性別の数値化と年齢のnull血の補間
df.Sex = df.Sex.replace(['male', 'female'], [0, 1])
df.fillna(df.Age.mean(), inplace=True)
df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
df.head(3)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,0,22.0,1,0,7.2500
1,1,1,1,38.0,1,0,71.2833
2,1,3,1,26.0,0,0,7.9250


In [4]:
#x,yに分割
x = df.copy()
x.drop(['Survived'], axis=1, inplace=True)
y = df.copy()
y = y['Survived']

In [5]:
#学習
model = RandomForestClassifier(n_estimators = 100)
model.fit(x, y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [6]:
#テストデータの読み込み、整形
test = pd.read_csv('../input/test.csv')
test.Sex = test.Sex.replace(['male', 'female'], [0, 1])
test.fillna(test.Age.mean(), inplace=True)
test.drop(['PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked'], axis=1, inplace=True)
test.head(3)

,Pclass,Sex,Age,SibSp,Parch,Fare
0,3,0,34.5,0,0,7.8292
1,3,1,47.0,1,0,7.0000
2,2,0,62.0,0,0,9.6875


In [7]:
# 提出用
output = model.predict(test)
df_out = pd.read_csv('../input/test.csv')
df_out['Survived'] = output
df_out = df_out[['PassengerId', 'Survived']]
df_out.to_csv('submission.csv', index=False)